<a href="https://colab.research.google.com/github/prabhavpratyaksh/Coursera_Capstone/blob/master/Week%203/Toronto%20Segmentation%20%26%20Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toronto Neigbourhoods Segmentation & Clustering

## Applied Data Science Capstone Project | Week 3 | Peer-Graded Assignment
## Prabhav Pratyaksh 26th July 2021

### Importing libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder
!pip install geopy
from geopy.geocoders import Nominatim 
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 98 kB 4.9 MB/s 


### Part 1 - Webscraping


In [17]:
# send request to Wikipedia link
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url).text

# using BeautifulSoup, get relevant tags for parsing
soup = BeautifulSoup(response, 'html5lib')
table = soup.find('table')
fields = table.find_all('td')
#print(soup.prettify())

df_1 = pd.DataFrame(columns=["Postal Code","Borough","Neighbourhood(s)"]) # creating a dataframe to add the values

# iterating through every column to separate the postal codes, boroughs, and the neigbourhoods
for cell in fields:
  if cell.span.text=='Not assigned':
        pass
  else:
      PC = cell.p.text[:3]
      BH = (cell.span.text).split('(')[0]
      NB = (((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
      df_1 = df_1.append({"Postal Code":PC,"Borough":BH,"Neighbourhood(s)":NB},ignore_index = True)

df_1['Borough']=df_1['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df_1.head()
print ("Dataframe has {} rows and {} columns".format(df_1.shape[0],df_1.shape[1]))


Dataframe has 103 rows and 3 columns


In [18]:
#Checking if there are any Neighbourhoods with Not Assigned value
df_1.filter(like = "Not assigned", axis = 0)

,Postal Code,Borough,Neighbourhood(s)


From the above code snippet, we see that there are no rows (both Boroughs and Neighbourhoods) that have Not Assigned value. This is because we already removed those while scraping

However, in case there were Boroughs with Not Assigned values, we could have used the below code to drop them from the dataframe

In [21]:
df_1.drop(df_1[df_1['Borough'] == 'Not assigned'].index, inplace=True)
print("Dataframe now has {} rows and {} columns".format(df_1.shape[0],df_1.shape[1]))
df_1.head()

Dataframe now has 103 rows and 3 columns


,Postal Code,Borough,Neighbourhood(s)
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Note that there are same number of rows and columns after dropping

Similarly, we could have used the below code to replace Neighbourhoods with Not assigned value with their respective Boroughs. However, as seen above there are no Boroughs or Neighbourhoods with Not assigned value

In [25]:
df_1.loc[df_1['Neighbourhood(s)'] == "Not assigned", "Neighbourhood(s)"] = df_1.loc[df_1['Neighbourhood(s)'] == "Not assigned", "Borough"]
df_1

,Postal Code,Borough,Neighbourhood(s)
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Now, we group Neighbourhoods by postal code

In [29]:

df_1 = df_1.groupby(['Postal Code', 'Borough'])['Neighbourhood(s)'].apply(', '.join).reset_index()
df_1.columns = ['Postal Code', 'Borough', 'Neighbourhood(s)']
df_1

,Postal Code,Borough,Neighbourhood(s)
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [30]:
print("Dataframe now has {} rows and {} columns".format(df_1.shape[0],df_1.shape[1]))

Dataframe now has 103 rows and 3 columns
